In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('./train.csv')
train = train[3387:]

In [3]:
test = pd.read_csv('./test.csv')
test

,mid,text
0,2dKrhwqWBVm_1195031270,蛔虫 美女 问世 一种 减肥 方法 不用 运动 不用 节食 吃 蛔虫 卵 服用 蛔虫 卵后 ...
1,2dKrzuu3skL_1644395354,传说 中 麦当劳 全国 通用 无线 上网 密码 真的 真的 假 还 伐 存 次 都 找 不到...
2,2dKs1KOT8t0_1314608344,顶花 黄瓜 猫腻 生长 剂 含 避孕药 青岛 新闻网 顶 鲜艳 黄花 黄瓜 市场 上 受欢迎...
3,2dKSSoZX2Sd_1300442112,临沂 市委书记 奸淫 上百 幼女 燕赵 都市网 原文 出处 人渣 无语 怒 抓狂 求证 真的...
4,2kiroqdeC1F_1764669250,更 亲近 人 发火 讨厌 人 亲近 人 大部分 时间 人 发火 是因为 亲近 人 更 期望 ...
...,...,...
1907,zzabOkjrC_1264341240,刚录 完 节目 评论 里 都 讨论 穿 平底鞋 穿 平底鞋 是因为 不想 小朋友 高太多 真...
1908,zzBmWlKN0_1725725780,自然 堂 全国 路演 随手 拍 自然 堂舞林 争霸 冠军 喜欢 张傲 一路 走来 都 支持 ...
1909,zzFTHcFe1_1628595353,回龙观 赶紧 检查一下 家里 牙膏 黑色 条 赶紧 扔掉 买 牙膏 时 牙膏管 反面 底部 ...
1910,zzj3JohLU_2683002893,買 牙膏 請 留心 买 牙膏 时 牙膏管 反面 底部 颜色 条 才 颜色 条有 含义 尽量 ...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.concat([train['text'], test['text']], axis=0)
vectorizer = TfidfVectorizer(tokenizer=str.split)
vectorizer = vectorizer.fit(test['text'])
X = vectorizer.transform(df)

d:\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:
X

<6138x228293 sparse matrix of type '<class 'numpy.float64'>'
	with 3531232 stored elements in Compressed Sparse Row format>

In [6]:
from sklearn.naive_bayes import MultinomialNB,ComplementNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import VotingClassifier

lr = LogisticRegression()
clf = ComplementNB(alpha=0.02)
sgd_model1 = SGDClassifier(max_iter=8000, tol=5e-4, loss="modified_huber")
sgd_model2 = SGDClassifier(max_iter=10000, tol=3e-4, loss="modified_huber", class_weight="balanced")
sgd_model3 = SGDClassifier(max_iter=15000, tol=1e-4, loss="modified_huber", early_stopping=True)

ensemble = VotingClassifier(
    estimators=[('lr', lr), ('cnb', clf), ('sgd1', sgd_model1), ('sgd2', sgd_model2), ('sgd3', sgd_model3)],
    weights=[0.20, 0.20, 0.20, 0.20, 0.20],
    voting='soft',
)
ensemble.fit(X[:train.shape[0]], train.is_rumor)

d:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\anaconda3\Lib\site-packages\s

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('cnb', ComplementNB(alpha=0.02)),
                             ('sgd1',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=8000, tol=0.0005)),
                             ('sgd2',
                              SGDClassifier(class_weight='balanced',
                                            loss='modified_huber',
                                            max_iter=10000, tol=0.0003)),
                             ('sgd3',
                              SGDClassifier(early_stopping=True,
                                            loss='modified_huber',
                                            max_iter=15000, tol=0.0001))],
                 voting='soft', weights=[0.2, 0.2, 0.2, 0.2, 0.2])

In [8]:
preds_test = ensemble.predict(X[train.shape[0]:])
preds_test

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [10]:
import joblib
joblib.dump(ensemble, 'ensemble_fit_in_test.pkl')

['ensemble_fit_in_test.pkl']

In [9]:
test["is_rumor"] = preds_test
test

,mid,text,is_rumor
0,2dKrhwqWBVm_1195031270,蛔虫 美女 问世 一种 减肥 方法 不用 运动 不用 节食 吃 蛔虫 卵 服用 蛔虫 卵后 ...,1
1,2dKrzuu3skL_1644395354,传说 中 麦当劳 全国 通用 无线 上网 密码 真的 真的 假 还 伐 存 次 都 找 不到...,1
2,2dKs1KOT8t0_1314608344,顶花 黄瓜 猫腻 生长 剂 含 避孕药 青岛 新闻网 顶 鲜艳 黄花 黄瓜 市场 上 受欢迎...,1
3,2dKSSoZX2Sd_1300442112,临沂 市委书记 奸淫 上百 幼女 燕赵 都市网 原文 出处 人渣 无语 怒 抓狂 求证 真的...,1
4,2kiroqdeC1F_1764669250,更 亲近 人 发火 讨厌 人 亲近 人 大部分 时间 人 发火 是因为 亲近 人 更 期望 ...,0
...,...,...,...
1907,zzabOkjrC_1264341240,刚录 完 节目 评论 里 都 讨论 穿 平底鞋 穿 平底鞋 是因为 不想 小朋友 高太多 真...,0
1908,zzBmWlKN0_1725725780,自然 堂 全国 路演 随手 拍 自然 堂舞林 争霸 冠军 喜欢 张傲 一路 走来 都 支持 ...,0
1909,zzFTHcFe1_1628595353,回龙观 赶紧 检查一下 家里 牙膏 黑色 条 赶紧 扔掉 买 牙膏 时 牙膏管 反面 底部 ...,1
1910,zzj3JohLU_2683002893,買 牙膏 請 留心 买 牙膏 时 牙膏管 反面 底部 颜色 条 才 颜色 条有 含义 尽量 ...,1


In [14]:
with open('submission.csv', 'w', encoding='utf-8') as f:
    f.write('id\tlabel\n')
    for _, row in test.iterrows():
        if row['text'] == '':
            f.write(f"{row['mid']}\t0\n")
        else:
            f.write(f"{row['mid']}\t{row['is_rumor']}\n")
